### Lib

In [19]:
import sys
from models import ASTModel
import dataloader
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import random
from collections import defaultdict
import argparse
from traintest import train, validate

sys.path.append('./src') 
os.environ['TORCH_HOME'] = '../pretrained_models'  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


### Data Spliting

In [20]:
def prepare_ast_5fold_patient_split(root_path, k_folds=5):
    # 1. จัดกลุ่มไฟล์ตามรหัสผู้ป่วย (Patient ID: 00x)
    patient_groups = {
        "Cough_PTB": defaultdict(list),
        "Cough_Non-PTB": defaultdict(list)
    }
    class_map = {"Cough_PTB": "1", "Cough_Non-PTB": "0"}

    # ดึงข้อมูลไฟล์ทั้งหมดเข้ากลุ่ม
    for folder_name, label_idx in class_map.items():
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.exists(folder_path): continue
        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                patient_id = file.split('_')[0]
                full_path = os.path.abspath(os.path.join(folder_path, file))
                patient_groups[folder_name][patient_id].append(full_path)

    # รวบรวมรายชื่อผู้ป่วยที่ไม่ซ้ำกันทั้งหมด
    all_patients = set()
    for folder_name in patient_groups:
        all_patients.update(patient_groups[folder_name].keys())
    all_patients = list(all_patients)
    
    # ล็อก Seed เพื่อให้ผลลัพธ์คงที่และสุ่มจัดเรียงผู้ป่วย
    random.seed(42)
    random.shuffle(all_patients)

    # แบ่งผู้ป่วยออกเป็น 5 กลุ่ม (Folds)
    fold_size = len(all_patients) // k_folds
    folds = []
    for i in range(k_folds):
        start_idx = i * fold_size
        # ให้ fold สุดท้ายรับเศษที่เหลือไปทั้งหมด (ถ้าหารไม่ลงตัว)
        end_idx = (i + 1) * fold_size if i < k_folds - 1 else len(all_patients)
        folds.append(all_patients[start_idx:end_idx])

    print("="*50)
    print("AST 5-FOLD CV DATA PREPARATION SUMMARY")
    print("="*50)

    # 2. สร้างไฟล์ JSON และนับสถิติในแต่ละ Fold
    for fold_idx, eval_ids in enumerate(folds, 1):
        # กลุ่ม Eval คือกลุ่มปัจจุบัน ส่วน Train คือกลุ่มที่เหลือทั้งหมด
        train_ids = [pid for pid in all_patients if pid not in eval_ids]
        
        train_list, eval_list = [], []
        stats = {
            "train": {"ptb": 0, "non_ptb": 0},
            "eval": {"ptb": 0, "non_ptb": 0}
        }

        for folder_name, groups in patient_groups.items():
            for p_id, paths in groups.items():
                for path in paths:
                    item = {"wav": path, "labels": class_map[folder_name]}
                    if p_id in eval_ids:
                        eval_list.append(item)
                        if class_map[folder_name] == "1": stats["eval"]["ptb"] += 1
                        else: stats["eval"]["non_ptb"] += 1
                    else:
                        train_list.append(item)
                        if class_map[folder_name] == "1": stats["train"]["ptb"] += 1
                        else: stats["train"]["non_ptb"] += 1

        # 3. บันทึกไฟล์ JSON ของแต่ละ Fold
        with open(f'train_data_fold_{fold_idx}.json', 'w') as f:
            json.dump({"data": train_list}, f, indent=4)
        with open(f'eval_data_fold_{fold_idx}.json', 'w') as f:
            json.dump({"data": eval_list}, f, indent=4)

        # 4. แสดงผลสรุปของ Fold นั้นๆ
        print(f"\n[ FOLD {fold_idx} ]")
        print(f"  TRAIN SET (Patients: {len(train_ids)} คน) -> IDs: {', '.join(sorted(train_ids))}")
        print(f"      Samples: PTB (Class 1) = {stats['train']['ptb']:3d} | Non-PTB (Class 0) = {stats['train']['non_ptb']:3d}")
        print(f"  EVAL SET  (Patients: {len(eval_ids)} คน) -> IDs: {', '.join(sorted(eval_ids))}")
        print(f"      Samples: PTB (Class 1) = {stats['eval']['ptb']:3d} | Non-PTB (Class 0) = {stats['eval']['non_ptb']:3d}")

# เรียกใช้งาน
prepare_ast_5fold_patient_split("./Data", k_folds=5)

AST 5-FOLD CV DATA PREPARATION SUMMARY

[ FOLD 1 ]
  TRAIN SET (Patients: 12 คน) -> IDs: 001, 002, 003, 005, 006, 007, 008, 009, 011, 012, 013, 016
      Samples: PTB (Class 1) = 150 | Non-PTB (Class 0) = 234
  EVAL SET  (Patients: 3 คน) -> IDs: 004, 014, 015
      Samples: PTB (Class 1) =  37 | Non-PTB (Class 0) =  91

[ FOLD 2 ]
  TRAIN SET (Patients: 12 คน) -> IDs: 001, 002, 003, 004, 006, 007, 008, 009, 011, 014, 015, 016
      Samples: PTB (Class 1) = 167 | Non-PTB (Class 0) = 253
  EVAL SET  (Patients: 3 คน) -> IDs: 005, 012, 013
      Samples: PTB (Class 1) =  20 | Non-PTB (Class 0) =  72

[ FOLD 3 ]
  TRAIN SET (Patients: 12 คน) -> IDs: 001, 002, 003, 004, 005, 008, 009, 012, 013, 014, 015, 016
      Samples: PTB (Class 1) = 134 | Non-PTB (Class 0) = 314
  EVAL SET  (Patients: 3 คน) -> IDs: 006, 007, 011
      Samples: PTB (Class 1) =  53 | Non-PTB (Class 0) =  11

[ FOLD 4 ]
  TRAIN SET (Patients: 12 คน) -> IDs: 001, 002, 004, 005, 006, 007, 009, 011, 012, 013, 014, 015
      

In [21]:

# สร้างตัวแปรเก็บผลลัพธ์ของแต่ละ Fold
fold_results = {"acc": [], "auc": []}

# วนลูป 5 Folds (1 ถึง 5)
for fold in range(1, 6):
    print(f"\n{'='*40}")
    print(f"🚀 STARTING FOLD {fold}/5")
    print(f"{'='*40}")

    # 1. กำหนดไฟล์ JSON ของ Fold ปัจจุบัน
    train_json = f'train_data_fold_{fold}.json'
    eval_json = f'eval_data_fold_{fold}.json'
    
    # กำหนดโฟลเดอร์สำหรับเก็บ Checkpoint แยกตาม Fold เพื่อไม่ให้ไฟล์ทับกัน
    exp_dir = f'./exp/tb_ast_p_fold_{fold}'
    os.makedirs(f'{exp_dir}/models', exist_ok=True)

    # 2. ตั้งค่า Data Config (อ้างอิงจากโค้ดเดิมของคุณ)
    train_audio_conf = {'num_mel_bins': 128, 'target_length': 100, 'freqm': 0, 'timem': 0, 'mixup': 0.0, 'dataset': 'audioset', 'mode': 'train', 'mean': -3.3831, 'std': 5.1156, 'noise': False, 'skip_norm': False}
    eval_audio_conf = {'num_mel_bins': 128, 'target_length': 100, 'freqm': 0, 'timem': 0, 'mixup': 0.0, 'dataset': 'audioset', 'mode': 'evaluation', 'mean': -3.3831, 'std': 5.1156, 'noise': False, 'skip_norm': False}
    
    BATCH_SIZE = 8
    
    # สร้าง DataLoader ของ Fold ปัจจุบัน
    train_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(train_json, label_csv='class_labels_indices.csv', audio_conf=train_audio_conf),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True
    )
    eval_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(eval_json, label_csv='class_labels_indices.csv', audio_conf=eval_audio_conf),
        batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True
    )

    # 3. สร้าง AST-P Model ใหม่ (ต้องสร้างใหม่ทุก Fold เพื่อล้างค่าน้ำหนักเดิม)
    ast_model = ASTModel(
        label_dim=2, 
        fstride=10, 
        tstride=10, 
        input_fdim=128, 
        input_tdim=100, 
        imagenet_pretrain=True, 
        audioset_pretrain=True,  # ใช้ AudioSet Pretrained (AST-P)
        model_size='base384'     # ขนาดที่รองรับสำหรับ AST-P
    )

    # 4. ตั้งค่า Training Arguments
    args = argparse.Namespace(
        exp_dir=exp_dir, dataset='audioset', n_class=2, lr=1e-5, n_epochs=30, batch_size=BATCH_SIZE, 
        n_print_steps=10, save_model=True, loss='CE', metrics='acc', 
        lrscheduler_start=10, lrscheduler_step=5, lrscheduler_decay=0.5, 
        warmup=True, wa=True, wa_start=1, wa_end=30
    )

    # 5. เริ่ม Train
    train(ast_model, train_loader, eval_loader, args)

    # 6. ประเมินผล (Validate) ด้วย Best Model ของ Fold นี้
    best_model_path = f'{args.exp_dir}/models/best_audio_model.pth'
    
    # โหลดโมเดลที่ดีที่สุดกลับมาทดสอบ
    best_model = ASTModel(label_dim=2, fstride=10, tstride=10, input_fdim=128, input_tdim=100, imagenet_pretrain=False, audioset_pretrain=False, model_size='base384', verbose=False)
    best_model = torch.nn.DataParallel(best_model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_model.load_state_dict(torch.load(best_model_path, map_location=device))
    
    stats, eval_loss = validate(best_model, eval_loader, args, epoch='best')
    acc = stats[0]['acc']
    auc = stats[0]['auc']
    
    # เก็บค่าสถิติไว้คำนวณเฉลี่ยตอนจบ
    fold_results["acc"].append(acc)
    fold_results["auc"].append(auc)

    print(f"\n[ RESULT FOLD {fold} ] Accuracy: {acc:.4f} | AUC: {auc:.4f}")

# 7. สรุปผลรวมแบบ Standard 5-Cross Validation
print(f"\n{'='*40}")
print("🏆 FINAL 5-FOLD CV RESULTS")
print(f"{'='*40}")
print(f"Averaged Accuracy : {np.mean(fold_results['acc']):.4f} ± {np.std(fold_results['acc']):.4f}")
print(f"Averaged AUC      : {np.mean(fold_results['auc']):.4f} ± {np.std(fold_results['auc']):.4f}")


🚀 STARTING FOLD 1/5
---------------the train dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -3.383 and std 5.116 to normalize the input.
number of classes is 2
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -3.383 and std 5.116 to normalize the input.
number of classes is 2
---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=108
running on cpu
Total parameter number is : 86.880 million
Total trainable parameter number is : 86.880 million
now training with audioset, main metrics: acc, loss function: CrossEntropyLoss(), learning rate scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001E284ECB820>
The learning rate scheduler starts at 10 epoch with decay r

z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\grad_scaler.py:116: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


warm-up learning rate is 0.000000


KeyboardInterrupt: 